<a href="https://colab.research.google.com/github/superpanditas/Quinio/blob/main/%5BForecasting_Units%5D_By_Stores_and_Items.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0 - Getting Started

**Business Problem:** A company is getting stock outs due to don't have enought ...

Help the Tech and Product Team to forecast units sold
The approach followed is begin to forecast units sold by all company, after by stores and finally by items. The goal is to forecast units sold by ...



In [ ]:
# @title Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

from sklearn.metrics import mean_squared_error

plt.style.use('ggplot')

# define the custom style function
def set_custom_style(df):
  return (df.style
          .format(precision='2', decimal='.', thousands=',')
          .set_table_styles({
                  '': [{'selector': 'td', 'props': [('background-color', '#f5f5f5')]}]
              })
          .set_properties(**{
            'font-size':'14px',
            'font-family':'Arial',
          })
          )

In [ ]:
# @title Load Data
df = pd.read_csv('/content/Productlevel_Sales_Transactions_Dataset_Weekly.csv')
df_styled = set_custom_style(df.head(5))
df_styled

# Step 1 - Data Cleaning


In [ ]:
# @title Reshaping DataFrame

'''
Transform the DataFrame to adjust it to analysis, where each row represents a single store,
a weekly and the units sold
'''

id_vars = ['Scode', 'Pcode', 'Price']
value_var = ['Wk'+str(i) for i in range(0,104)]
var_name = 'weekly'
value_name = 'units'

# rename columns
new_col_names = ['store', 'item', 'price', 'weekly', 'units']

# compute melt function to change the data format
sales = df.melt(id_vars=id_vars, value_vars=value_var, var_name=var_name, value_name=value_name)
sales.columns = new_col_names

In [ ]:
# @title Duplicate Values

'''
The dataframe has different prices for some values ​​['weekly', 'store', 'price'], ...
for simplicity, we will sum all units by 'weekly', 'store' and 'price' columns and omit the unit price column
'''

sales = sales.groupby(['weekly', 'store', 'item'])['units'].sum().reset_index()

In [ ]:
# @title Weekly Dates

'''
It's more useful have real weekly dates, so we're going to add a column of dates
'''

# initialize of first weekly date (Note that weeks start on sundays)
init_week = pd.to_datetime('2023-01-07')

# create a function to compute mapping from weekly column to weekly dates according to the number of week
def calculate_date(weekly):
  week_number = int(weekly[2:])
  return init_week + timedelta(weeks=week_number)

sales['weekly_date'] = sales['weekly'].apply(calculate_date)

In [ ]:
# @title Weekly and Monthly (DataFrame)

'''
There'are multiple prices for some items in the same timeframe, so for simplicity
we're going to eliminate 'price' column and sum units.
'''

# create the dataframe called 'weekly_sales_by_store'
weekly_sales_by_store = sales.copy()

# drop "weekly" column
weekly_sales_by_store = weekly_sales_by_store.drop('weekly', axis=1)

# data grouped
weekly_sales_by_store = weekly_sales_by_store.groupby(['weekly_date', 'store', 'item'])['units'].sum().reset_index()

# set index 'weekly_date' column
weekly_sales_by_store = weekly_sales_by_store.set_index('weekly_date')
weekly_sales_by_store.index = pd.to_datetime(weekly_sales_by_store.index)

# -------

# create the dataframe called 'monthly_sales_by_store' using resample function

monthly_sales_by_store = weekly_sales_by_store.copy()
# compute resample function
monthly_sales_by_store = monthly_sales_by_store.groupby(['store', 'item']).resample('M')['units'].sum().reset_index()

# set and rename index 'weekly_date' column to 'monthly date'
monthly_sales_by_store = monthly_sales_by_store.set_index('weekly_date')
monthly_sales_by_store = monthly_sales_by_store.rename_axis('monthly_date')


# Step 2 - Data Understanding

Analysis Highlights:
  
  Weekly Units (From Jan to Oct): 2023 vs 2024
    - The units sold WoW keep similar behavior between Jan-Oct but Nov-Dec ...

In [ ]:
# @title Monthly and Weekly Stats (DataFrame)
# create 'df_monthly' and 'df_weekly' dataframe to calculate stats
df_monthly = monthly_sales_by_store.copy()
df_weekly = weekly_sales_by_store.copy()

# add 'year' and 'month' column
df_monthly['year'] = df_monthly.index.year
df_monthly['month'] = df_monthly.index.month

df_weekly['year'] = df_weekly.index.year
df_weekly['month'] = df_weekly.index.month

# -------
# create 'monthly_stats' dataframe
monthly_stats = df_monthly.groupby(['monthly_date']).agg({
    'units': [lambda x: np.percentile(x, 25), 'median', lambda x: np.percentile(x, 95), 'sum', 'max'],
    'item': 'nunique'
})

monthly_stats.columns = ['25th_perc', 'median', '95th_perc', 'units', 'max', 'total_items']
monthly_stats['pct_change'] = np.round(100 * monthly_stats['units'].pct_change(12),2)
monthly_stats['diff'] = np.round(monthly_stats['units'].diff(12))

# -------
# create 'monthly_stats_by_store' dataframe
monthly_stats_by_store = df_monthly.groupby(['monthly_date', 'store']).agg({
    'units': [lambda x: np.percentile(x, 25), 'median', lambda x: np.percentile(x, 95), 'sum', 'max'],
    'item': 'nunique'
})

monthly_stats_by_store.columns = ['25th_perc', 'median', '95th_perc', 'units', 'max', 'total_items']
monthly_stats_by_store['pct_change'] = np.round(100 * monthly_stats_by_store.groupby('store')['units'].pct_change(12),2)
monthly_stats_by_store['diff'] = np.round(monthly_stats_by_store.groupby('store')['units'].diff(12))

monthly_stats_by_store = monthly_stats_by_store.reset_index('store')

# -------
# create 'weekly_stats' dataframe
weekly_stats = df_weekly.groupby(['weekly_date']).agg({
    'units': [lambda x: np.percentile(x, 25), 'median', lambda x: np.percentile(x, 95), 'sum', 'max'],
    'item': 'nunique'
})

weekly_stats.columns = ['25th_perc', 'median', '95th_perc', 'units', 'max', 'total_items']
weekly_stats['pct_change'] = np.round(100 * weekly_stats['units'].pct_change(52),2)
weekly_stats['diff'] = np.round(weekly_stats['units'].diff(52))

# -------
# create 'weekly_stats_by_store' dataframe
weekly_stats_by_store = df_weekly.groupby(['weekly_date', 'store']).agg({
    'units': [lambda x: np.percentile(x, 25), 'median', lambda x: np.percentile(x, 95), 'sum', 'max', 'mean'],
    'item': 'nunique'
})

weekly_stats_by_store.columns = ['25th_perc', 'median', '95th_perc', 'units', 'max', 'total_items', 'mean']
weekly_stats_by_store['pct_change'] = np.round(100 * weekly_stats_by_store.groupby('store')['units'].pct_change(52),2)
weekly_stats_by_store['diff'] = np.round(weekly_stats_by_store.groupby('store')['units'].diff(52))

weekly_stats_by_store = weekly_stats_by_store.reset_index('store')

# -------
# create 'weekly_stats_by_store_and_item' dataframe
weekly_stats_by_store_and_item = df_weekly.groupby(['weekly_date', 'store', 'item']).agg({
    'units': 'sum'
})

weekly_stats_by_store_and_item.columns = ['units']
weekly_stats_by_store_and_item['pct_change'] = np.round(100 * weekly_stats_by_store_and_item.groupby(['store', 'item'])['units'].pct_change(52),2)
weekly_stats_by_store_and_item['diff'] = np.round(weekly_stats_by_store_and_item.groupby(['store', 'item'])['units'].diff(52))
weekly_stats_by_store_and_item['lag_52'] = np.round(weekly_stats_by_store_and_item.groupby(['store', 'item'])['units'].shift(52))

weekly_stats_by_store_and_item = weekly_stats_by_store_and_item.reset_index('store')
weekly_stats_by_store_and_item = weekly_stats_by_store_and_item.reset_index('item')

In [ ]:
def timeseries_plot(df, date_format, year_one, year_two, start_month, end_month, y_lim, title='weekly'):

  start_date_one = year_one + '-' + start_month
  end_date_one = year_one + '-' + end_month

  start_date_two = year_two + '-' + start_month
  end_date_two = year_two + '-' + end_month


  title_one = title + ' ' + 'Units Sold (2023): 25th, 50th and 95th Percentile'
  title_two = title + ' ' + 'Units Sold (2024): 25th, 50th and 95th Percentile'

  y_lim_one = y_lim
  y_lim_two = y_lim

  date_format = date_format
  stats = df

  # -------

  fig, ax = plt.subplots(2, 1, sharey=False, figsize=(15, 14))

  ax[0].set_title(title_one)
  ax[0].plot(stats[start_date_one:end_date_one].index.strftime(date_format),
            stats[start_date_one:end_date_one]['median'])
  ax[0].plot(stats[start_date_one:end_date_one].index.strftime(date_format),
            stats[start_date_one:end_date_one]['25th_perc'],
            linestyle='--')
  ax[0].plot(stats[start_date_one:end_date_one].index.strftime(date_format),
            stats[start_date_one:end_date_one]['95th_perc'],
            linestyle='--')
  ax[0].set_ylim(0, y_lim_one)
  ax[0].tick_params(axis='x', rotation=90)

  ax[1].set_title(title_two)
  ax[1].plot(stats[start_date_two:end_date_two].index.strftime(date_format),
            stats[start_date_two:end_date_two]['median'])
  ax[1].plot(stats[start_date_two:end_date_two].index.strftime(date_format),
            stats[start_date_two:end_date_two]['25th_perc'],
            linestyle='--')
  ax[1].plot(stats[start_date_two:end_date_two].index.strftime(date_format),
            stats[start_date_two:end_date_two]['95th_perc'],
            linestyle='--')
  ax[1].set_ylim(0, y_lim_two)
  ax[1].tick_params(axis='x', rotation=90)

  plt.show()

In [ ]:
# @title  Weekly Units Sold (Jan-Nov): 2023 vs 2024
timeseries_plot(weekly_stats, '%b-%d', '2023', '2024', '01-01', '11-25', 160, 'Weekly')

In [ ]:
# @title Weekly Units Sold (Nov-Dic): 2023 vs 2024
timeseries_plot(weekly_stats, '%b-%d', '2023', '2024', '11-23', '12-31', 4000, 'Weekly')

In [ ]:
# @title Weekly Units Sold: YoY Pct Change (%)
fig, ax = plt.subplots(1, 2, figsize=(20, 4))

ax[0].set_title('Weekly Units (Jan-Nov): YoY Pct Change (%)')
ax[0].plot(weekly_stats['2024-01-01':'2024-11-18'].index.strftime('%b-%d'), weekly_stats['2024-01-01':'2024-11-18']['pct_change'])
ax[0].tick_params(axis='x', rotation=90)
ax[0].set_ylim(0, 5)

ax[1].set_title('Weekly Units Sold (Nov-Dic): YoY Pct Change (%)')
ax[1].plot(weekly_stats['2024-11-18':'2024-12-31'].index.strftime('%b-%d'), weekly_stats['2024-11-18':'2024-12-31']['pct_change'])
ax[1].tick_params(axis='x', rotation=90)

plt.show()

In [ ]:
weekly_stats_by_store_one = weekly_stats_by_store[weekly_stats_by_store['store'] == 'Store1']
weekly_stats_by_store_two = weekly_stats_by_store[weekly_stats_by_store['store'] == 'Store2']
weekly_stats_by_store_three = weekly_stats_by_store[weekly_stats_by_store['store'] == 'Store3']
weekly_stats_by_store_four = weekly_stats_by_store[weekly_stats_by_store['store'] == 'Store4']
weekly_stats_by_store_five = weekly_stats_by_store[weekly_stats_by_store['store'] == 'Store5']

# weekly_stats_by_store_and_item_one = weekly_stats_by_store_and_item[weekly_stats_by_store_and_item['store'] == 'Store2']
# weekly_stats_by_store_and_item_one[weekly_stats_by_store_and_item_one['lag_52'] == 0]

data = weekly_stats_by_store_and_item[weekly_stats_by_store_and_item['lag_52'] == 0]
data_agg = data.groupby(['weekly_date', 'store'])['units'].sum().reset_index()
data_agg = data_agg.set_index('weekly_date')

# data_agg.head()
store1 = weekly_stats_by_store_and_item[(weekly_stats_by_store_and_item['lag_52'] > 0) & (weekly_stats_by_store_and_item['store'] == 'Store1') & (weekly_stats_by_store_and_item.index == '2024-07-06')]
store1.sort_values('diff', ascending=False).head(50)

# data_agg[data_agg['store'] == 'Store1']['units'].plot()

In [ ]:
# @title Weekly Units Sold By Store: YoY Pct Change (%)
fig, ax = plt.subplots(5, 2, figsize=(20, 35))

# ------- Store 1
ax[0,0].set_title('Weekly Units Sold - Store 1 (Jan-Nov): YoY Pct Change (%)')
ax[0,0].plot(weekly_stats_by_store_one['2024-01-01':'2024-11-18'].index.strftime('%b-%d'), weekly_stats_by_store_one['2024-01-01':'2024-11-18']['pct_change'])
ax[0,0].tick_params(axis='x', rotation=90)
# ax[0,0].set_ylim(0, 5)

ax[0,1].set_title('Weekly Units Sold - Store 1 (Nov-Dic): YoY Pct Change (%)')
ax[0,1].plot(weekly_stats_by_store_one['2024-11-18':'2024-12-31'].index.strftime('%b-%d'), weekly_stats_by_store_one['2024-11-18':'2024-12-31']['pct_change'])
ax[0,1].tick_params(axis='x', rotation=90)

# ------- Store 2
ax[1,0].set_title('Weekly Units Sold - Store 2 (Jan-Nov): YoY Pct Change (%)')
ax[1,0].plot(weekly_stats_by_store_two['2024-01-01':'2024-11-18'].index.strftime('%b-%d'), weekly_stats_by_store_two['2024-01-01':'2024-11-18']['pct_change'])
ax[1,0].tick_params(axis='x', rotation=90)

ax[1,1].set_title('Weekly Units Sold - Store 2 (Nov-Dic): YoY Pct Change (%)')
ax[1,1].plot(weekly_stats_by_store_two['2024-11-18':'2024-12-31'].index.strftime('%b-%d'), weekly_stats_by_store_two['2024-11-18':'2024-12-31']['pct_change'])
ax[1,1].tick_params(axis='x', rotation=90)

# ------- Store 3
ax[2,0].set_title('Weekly Units Sold - Store 3 (Jan-Nov): YoY Pct Change (%)')
ax[2,0].plot(weekly_stats_by_store_three['2024-01-01':'2024-11-18'].index.strftime('%b-%d'), weekly_stats_by_store_three['2024-01-01':'2024-11-18']['pct_change'])
ax[2,0].tick_params(axis='x', rotation=90)

ax[2,1].set_title('Weekly Units Sold - Store 3 (Nov-Dic): YoY Pct Change (%)')
ax[2,1].plot(weekly_stats_by_store_three['2024-11-18':'2024-12-31'].index.strftime('%b-%d'), weekly_stats_by_store_three['2024-11-18':'2024-12-31']['pct_change'])
ax[2,1].tick_params(axis='x', rotation=90)

# ------- Store 4
ax[3,0].set_title('Weekly Units Sold - Store 4 (Jan-Nov): YoY Pct Change (%)')
ax[3,0].plot(weekly_stats_by_store_four['2024-01-01':'2024-11-18'].index.strftime('%b-%d'), weekly_stats_by_store_four['2024-01-01':'2024-11-18']['pct_change'])
ax[3,0].tick_params(axis='x', rotation=90)

ax[3,1].set_title('Weekly Units Sold - Store 4 (Nov-Dic): YoY Pct Change (%)')
ax[3,1].plot(weekly_stats_by_store_four['2024-11-18':'2024-12-31'].index.strftime('%b-%d'), weekly_stats_by_store_four['2024-11-18':'2024-12-31']['pct_change'])
ax[3,1].tick_params(axis='x', rotation=90)

# ------- Store 5
ax[4,0].set_title('Weekly Units Sold - Store 5 (Jan-Nov): YoY Pct Change (%)')
ax[4,0].plot(weekly_stats_by_store_five['2024-01-01':'2024-11-18'].index.strftime('%b-%d'), weekly_stats_by_store_five['2024-01-01':'2024-11-18']['pct_change'])
ax[4,0].tick_params(axis='x', rotation=90)

ax[4,1].set_title('Weekly Units Sold - Store 5 (Nov-Dic): YoY Pct Change (%)')
ax[4,1].plot(weekly_stats_by_store_five['2024-11-18':'2024-12-31'].index.strftime('%b-%d'), weekly_stats_by_store_five['2024-11-18':'2024-12-31']['pct_change'])
ax[4,1].tick_params(axis='x', rotation=90)

plt.show()

In [ ]:
timeseries_plot(weekly_stats_by_store[weekly_stats_by_store['store'] == 'Store3'], '%b-%d', '2023', '2024', '01-01', '11-25', 700, 'Weekly')

# Step 3 - Feauture Engineering

- Lags
- Seasonality
- Month
- ...

In [ ]:
# -------
'''num_lags = 13
window_size = 3

for lag_n in range(7, num_lags):
  # create the lag_12 column by grouping by store
  company_sales_by_monthly[f'lag_{lag_n}'] = company_sales_by_monthly.groupby(['store'])['units'].shift(lag_n)

  # compute EWMA for the 'units' column and use it to fill NaN values in the 'lag_12' column
  ewma_filled_values = company_sales_by_monthly.groupby('store')['units'].transform(lambda x: x.ewm(span=window_size, adjust=False).mean())

  # fill NaN values in the 'lag_12' column using the computed EWMA values
  company_sales_by_monthly[f'lag_{lag_n}'] = company_sales_by_monthly[f'lag_{lag_n}'].fillna(np.round(ewma_filled_values,2))'''
# -------


In [ ]:
# Autocorrelation Plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

plot_pacf(weekly_stats_by_store_and_item[(weekly_stats_by_store_and_item['item'] == 'SKU1') & (weekly_stats_by_store_and_item['store'] == 'Store2')].units, lags=52, alpha=0.05)
pacf(weekly_stats_by_store_and_item[(weekly_stats_by_store_and_item['item'] == 'SKU1') & (weekly_stats_by_store_and_item['store'] == 'Store2')].units, nlags=52)
# weekly_stats_by_store_and_item[(weekly_stats_by_store_and_item['item'] == 'SKU1') & (weekly_stats_by_store_and_item['store'] == 'Store1')]

In [ ]:
# @title Lags Feature

lags = [1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 50, 51, 52]

# compute lags
for lag_n in lags:
  weekly_sales_by_store[f'lag_{lag_n}'] = weekly_sales_by_store.groupby(['store', 'item'])['units'].shift(lag_n)

# fill NaN values
weekly_sales_by_store['lag_52'] = weekly_sales_by_store['lag_52'].fillna(weekly_sales_by_store['lag_1'])
weekly_sales_by_store['lag_51'] = weekly_sales_by_store['lag_51'].fillna(weekly_sales_by_store['lag_2'])
weekly_sales_by_store['lag_50'] = weekly_sales_by_store['lag_50'].fillna(weekly_sales_by_store['lag_3'])

# weekly_sales_by_store[(weekly_sales_by_store['store'] == 'Store5') & (weekly_sales_by_store['item'] == 'SKU99')].head(54)

In [ ]:
# @title Peak Weeks Feauture

# add seasonal pattern column
weekly_sales_by_store.loc[weekly_sales_by_store.index.isin([
    '2023-12-02',
    '2023-12-09',
    '2023-12-16',
    '2023-12-23',
    '2024-11-23',
    '2024-11-30',
    '2024-12-07',
    '2024-12-14']), 'is_peak_season'] = 1

weekly_sales_by_store.loc[~weekly_sales_by_store.index.isin([
    '2023-12-02',
    '2023-12-09',
    '2023-12-16',
    '2023-12-23',
    '2024-11-23',
    '2024-11-30',
    '2024-12-07',
    '2024-12-14']), 'is_peak_season'] = 0

In [ ]:
# @title Timeframe Feature
# cast index as DatetimeIndex
weekly_sales_by_store.index = pd.to_datetime(weekly_sales_by_store.index)

# add week column
weekly_sales_by_store['week_number'] = weekly_sales_by_store.index.to_series().dt.isocalendar().week

# sort the DatetimeIndex
weekly_sales_by_store = weekly_sales_by_store.sort_index()

In [ ]:
'''
# add difference from 12 months ago
company_sales_by_monthly['diff_12'] = company_sales_by_monthly.groupby('store')['units'].diff(12)

# compute diff_1 for the 'units' column and use it to fill NaN values in the 'diff_12' column
diff_filled_values = company_sales_by_monthly.groupby('store')['units'].transform(lambda x: x.diff(1))

# fill NaN values in the 'diff_12' column using the computed diff_1 values
company_sales_by_monthly['diff_12'] = company_sales_by_monthly['diff_12'].fillna(np.round(diff_filled_values,2))
'''

In [ ]:
# @title Encode categorical variables
# Combine 'store' and 'item' into a single column
weekly_sales_by_store['store_item'] = weekly_sales_by_store['store'] + '_' + weekly_sales_by_store['item']

weekly_sales_by_store = weekly_sales_by_store[weekly_sales_by_store['store'].isin(['Store2', 'Store3', 'Store4', 'Store5'])]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
weekly_sales_by_store['store_item_encoded'] = le.fit_transform(weekly_sales_by_store['store_item'])
weekly_sales_by_store.head()


# Step 4 - Data Modeling

In [ ]:
#weekly_sales_by_store = weekly_sales_by_store[weekly_sales_by_store['store'] == 'Store2']
weekly_sales_by_store = weekly_sales_by_store.reset_index()
weekly_sales_by_store = weekly_sales_by_store.set_index(['weekly_date', 'store', 'item'])
weekly_sales_by_store.head()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

data = weekly_sales_by_store.drop(['lag_1',
                                   'lag_2',
                                   'lag_3',
                                   'store_item'], axis=1)

FEATURE = ['lag_50',
           'lag_51',
           'lag_52',
           'lag_14',
           'lag_13',
           'lag_12',
           'lag_11',
           'lag_10',
           'lag_9',
           'lag_8',
           'lag_7',
           'is_peak_season',
           'week_number',
           'store_item_encoded']

TARGET = ['units']

# weekly_sales_by_monthly = weekly_sales_by_monthly.set_index('weekly_date')
data_train = data['2023-11-01':'2024-11-16']
data_test = data['2024-11-16':'2024-12-31']

x_train, y_train = data_train[FEATURE], data_train[TARGET]

x_test, y_test = data_test[FEATURE], data_test[TARGET]


param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2],
    'lambda': [0, 1],
    'alpha': [0, 1]
}

random_search = RandomizedSearchCV(estimator=xgb.XGBRegressor(),
                                   param_distributions=param_dist,
                                   n_iter=50,
                                   cv=3,
                                   scoring='neg_mean_squared_error',
                                   random_state=5)

random_search.fit(x_train, y_train)
print("Best Parameters: ", random_search.best_params_)

In [ ]:
# best parameters
best_params = random_search.best_params_
# best model
model_best = xgb.XGBRegressor(**best_params)
# train model
model_best.fit(x_train, y_train)
# forecasting ...
y_pred = model_best.predict(x_test)

# Step 5 - Data Evaluation

In [ ]:
print("Best Model MSE: ", mean_squared_error(y_test, y_pred))
print("Best Model RMSE: ", np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
# Get feature importance
feature_importances = model_best.feature_importances_

# Create a DataFrame to display the feature importances
importance_df = pd.DataFrame({
    'Feature': x_train.columns,
    'Importance': feature_importances
})

importance_df.sort_values('Importance', ascending=False).head(10)

In [ ]:
forecasted_sales = data_test.copy()

forecasted_sales['pred'] = np.round(y_pred, 0)
forecasted_sales['error'] = np.round(forecasted_sales['pred'] - forecasted_sales['units'], 2)
forecasted_sales = forecasted_sales.groupby(['weekly_date', 'store', 'item']).agg({
    'units':'sum',
    'lag_52':'sum',
    'pred':'sum',
    'error':'sum'
})

# forecasting sales by weekly_date and store columns
forecasted_sales_by_store = forecasted_sales.groupby(['weekly_date', 'store']).agg({
    'units':'sum',
    'lag_52':'sum',
    'pred':'sum'
})

# forecasting sales by weekly_date column
forecasted_sales_by_week = forecasted_sales.groupby(['weekly_date']).agg({
    'units':'sum',
    'lag_52':'sum',
    'pred':'sum'
})

# calculate pct_error column in the forecasted_sales dataframe
forecasted_sales['pct_error'] = np.round(100 * ((forecasted_sales['pred']/forecasted_sales['units']) - 1), 2)

# calculate error and pct_error columns in the forecasted_sales_by_store dataframe
forecasted_sales_by_store['error'] = np.round(forecasted_sales_by_store['pred'] - forecasted_sales_by_store['units'], 2)
forecasted_sales_by_store['pct_error'] = np.round( 100 * ((forecasted_sales_by_store['pred']/forecasted_sales_by_store['units']) - 1), 2)
# calculate error and pct_error columns in the forecasted_sales_by_week dataframe
forecasted_sales_by_week['error'] = np.round(forecasted_sales_by_week['pred'] - forecasted_sales_by_week['units'], 2)
forecasted_sales_by_week['pct_error'] = np.round( 100 * ((forecasted_sales_by_week['pred']/forecasted_sales_by_week['units']) - 1), 2)

In [ ]:
print('------- Forecast Units Sold by Item -------')
forecasted_sales_style = set_custom_style(forecasted_sales.head(5))
forecasted_sales_style

In [ ]:
over = 30
under = -30

overestimated = forecasted_sales['pct_error'] <= over
underestimated = forecasted_sales['pct_error'] >= under

good_performance = forecasted_sales[(overestimated) & (underestimated)]
bad_performance = forecasted_sales[~((overestimated) & (underestimated))]

good_performance = good_performance.reset_index()
good_performance = good_performance.groupby(['weekly_date', 'store']).agg({
    'item':'nunique',
    'units':'sum',
    'lag_52':'sum',
    'pred':'sum'
})

bad_performance = bad_performance.reset_index()
bad_performance = bad_performance.groupby(['weekly_date', 'store']).agg({
    'item':'nunique',
    'units':'sum',
    'lag_52':'sum',
    'pred':'sum'
})

df_performance = good_performance.join(bad_performance, how='right', rsuffix='_2')

df_performance.head(55)

In [ ]:
# @title Forecast Units Sold by Store
print('------- Forecast Units Sold by Store -------')
print('------- SEP/24 -------')
print('RMSE Store2: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store2'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store2'], 'pred'])), 2))
print('RMSE Store3: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store3'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store3'], 'pred'])), 2))
print('RMSE Store4: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store4'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store4'], 'pred'])), 2))
print('RMSE Store5: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store5'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-09-01':'2024-09-30',
                                                                                                              'Store5'], 'pred'])), 2))
print('------- OCT/24 -------')
print('RMSE Store2: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store2'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store2'], 'pred'])), 2))
print('RMSE Store3: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store3'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store3'], 'pred'])), 2))
print('RMSE Store4: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store4'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store4'], 'pred'])), 2))
print('RMSE Store5: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store5'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-10-01':'2024-10-31',
                                                                                                              'Store5'], 'pred'])), 2))
print('------- NOV/24 -------')
print('RMSE Store2: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store2'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store2'], 'pred'])), 2))
print('RMSE Store3: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store3'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store3'], 'pred'])), 2))
print('RMSE Store4: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store4'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store4'], 'pred'])), 2))
print('RMSE Store5: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store5'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-11-01':'2024-11-30',
                                                                                                              'Store5'], 'pred'])), 2))
print('------- DIC/24 -------')
print('RMSE Store2: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store2'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store2'], 'pred'])), 2))
print('RMSE Store3: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store3'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store3'], 'pred'])), 2))
print('RMSE Store4: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store4'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store4'], 'pred'])), 2))
print('RMSE Store5: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store5'], 'units'],
                                                    forecasted_sales_by_store.loc[pd.IndexSlice['2024-12-01':'2024-12-31',
                                                                                                              'Store5'], 'pred'])), 2))
forecasted_sales_by_store_style = set_custom_style(forecasted_sales_by_store['2024-11-01':'2024-11-30'].head(5))
forecasted_sales_by_store_style


In [ ]:
avg_weekly_sales_per_month = forecasted_sales_by_store.reset_index()
avg_weekly_sales_per_month['weekly_date'] = pd.to_datetime(avg_weekly_sales_per_month['weekly_date'])
avg_weekly_sales_per_month['month'] = avg_weekly_sales_per_month['weekly_date'].dt.month

avg_weekly_sales_per_month = avg_weekly_sales_per_month.groupby(['store', 'month']).agg({
    'units':'mean'
})

avg_weekly_sales_per_month.head()

In [ ]:
print('------- Forecast Units Sold by Week -------')
# print('RMSE SEP/24: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_week['2024-09-01':'2024-09-30'].units,
#                                                     forecasted_sales_by_week['2024-09-01':'2024-09-30'].pred)), 2))
# print('RMSE OCT/24: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_week['2024-10-01':'2024-10-31'].units,
#                                                     forecasted_sales_by_week['2024-10-01':'2024-10-31'].pred)), 2))
print('RMSE NOV/24: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_week['2024-11-01':'2024-11-30'].units,
                                                    forecasted_sales_by_week['2024-11-01':'2024-11-30'].pred)), 2))
print('RMSE DIC/24: ', np.round(np.sqrt(mean_squared_error(forecasted_sales_by_week['2024-12-01':'2024-12-31'].units,
                                                    forecasted_sales_by_week['2024-12-01':'2024-12-31'].pred)), 2))

forecasted_sales_by_week_style = set_custom_style(forecasted_sales_by_week.head(15))
forecasted_sales_by_week_style

In [ ]:
def plot(df):
  df['units'].plot(color='blue', label='units')
  df['pred'].plot(color='red', label='forecast')

  plt.legend()
  plt.show()

In [ ]:
plot(forecasted_sales_by_week['2024-11-01':'2024-12-31'])

# Step 6 - Next Steps

In [ ]:
sales_by_store_flt = sales_by_store[sales_by_store['store'] == 'Store4']

# Apply Exponential Moving Average
sales_by_store_flt['ema'] = sales_by_store_flt['units'].ewm(span=5, adjust=False).mean()

# Plot the smoothed time series
sales_by_store_flt[['units', 'ema']].plot(figsize=(10, 6), title='Time Series with EMA')
plt.show()



# fig, ax = plt.subplots()

# sales_by_store_temp = sales_by_store[sales_by_store['store'] == 'Store1']
# sales_by_store_temp = sales_by_store_temp["2025-10-01":"2025-12-28"]

# ax.plot(sales_by_store_temp.index, sales_by_store_temp['units'])
# plt.show()

In [ ]:
'''
Step 0 - Getting Started
- Import Libraries
- Load Data

Step 1 - Data Cleaning
- Change Wide to Long Format
- Change Data Types
- Map Categorical Variables To Weekly Dates

Step 2 - Data Understanding
- Stats
- Plot ACF

Step 3 - Feature Engineering
- Lags
- Seasonal Pattern
- Month Encoded
- Store Encoded

Step 4 - Data Modeling
- Split Train and Test
- GridSearch
- Model

Step 5 - Data Evaluation
- RMSE

Step 6 - Next Steps
-
'''